In [0]:
print("UDF Assignment")

UDF Assignment


In [0]:
# Databricks notebook source
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, udf
from pyspark.sql.types import IntegerType,DoubleType
spark = SparkSession.builder.appName("Spark DataFrames").getOrCreate()

# COMMAND ----------
df = spark.read.options(header='True', inferSchema='True').csv('/FileStore/tables/OfficeData.csv')
df.show()

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        James|     Sales|   NY| 90000| 34|10000|
|      Michael|     Sales|   NY| 86000| 56|20000|
|       Robert|     Sales|   CA| 81000| 30|23000|
|        Maria|   Finance|   CA| 90000| 24|23000|
|        Raman|   Finance|   CA| 99000| 40|24000|
|        Scott|   Finance|   NY| 83000| 36|19000|
|          Jen|   Finance|   NY| 79000| 53|15000|
|         Jeff| Marketing|   CA| 80000| 25|18000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
+-------------+----------+-----+------+---+-----+



In [0]:
def Increment(state,salary,bonus):
  if state =='NY':
    return (salary*0.1 + bonus*0.05)
  elif state == 'CA':
    return (salary*0.12 + bonus*0.03)
  else:
    return 0

In [0]:
Increment = udf(lambda x,y,z : Increment(x,y,z), DoubleType())
df.withColumn("Increment", Increment(df.state,df.salary,df.bonus)).show()

---------------------------------------------------------------------------
PythonException                           Traceback (most recent call last)
<command-1699698136113742> in <module>
      1 Increment = udf(lambda x,y,z : Increment(x,y,z), DoubleType())
----> 2 df.withColumn("Increment", Increment(df.state,df.salary,df.bonus)).show()

/databricks/spark/python/pyspark/sql/dataframe.py in show(self, n, truncate, vertical)
    500 
    501         if isinstance(truncate, bool) and truncate:
--> 502             print(self._jdf.showString(n, 20, vertical))
    503         else:
    504             try:

/databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1302 
   1303         answer = self.gateway_client.send_command(command)
-> 1304         return_value = get_return_value(
   1305             answer, self.gateway_client, self.target_id, self.name)
   1306 

/databricks/spark/python/pyspark/sql/utils.py in deco(*a, **kw)
    121       

In [0]:
# We need to make sure that the UDF variable should not be same with the function name, else the above error is thrown.
# The correct approach is as below :

Increment_Udf = udf(lambda x,y,z : Increment(x,y,z), DoubleType())


In [0]:
df.withColumn("Increment", Increment_Udf(df.state,df.salary,df.bonus)).show()

Traceback (most recent call last):
  File "/databricks/spark/python/pyspark/serializers.py", line 473, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "/databricks/spark/python/pyspark/cloudpickle/cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "/databricks/spark/python/pyspark/cloudpickle/cloudpickle_fast.py", line 563, in dump
    return Pickler.dump(self, obj)
TypeError: cannot pickle '_thread.RLock' object


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
/databricks/spark/python/pyspark/serializers.py in dumps(self, obj)
    472         try:
--> 473             return cloudpickle.dumps(obj, pickle_protocol)
    474         except pickle.PickleError:

/databricks/spark/python/pyspark/cloudpickle/cloudpickle_fast.py in dumps(obj, protocol, buffer_callback)
     72             )
---> 73             cp.dump(obj)
     74             return file.getvalue()

/databricks/spark/python/pyspark/cloudpickle/cloudpickle_fast.py in dump(self, obj)
    562         try:
--> 563             return Pickler.dump(self, obj)
    564         except RuntimeError as e:

TypeError: cannot pickle '_thread.RLock' object

During handling of the above exception, another exception occurred:

PicklingError                             Traceback (most recent call last)
<command-1699698136113745> in <module>
----> 1 df.

In [0]:
# The above error will come, if the function definition is not executed
# We need to create and execute function before, we create the udf object

In [0]:
df.withColumn("Increment", Increment_Udf(df.state,df.salary,df.bonus)).show()

+-------------+----------+-----+------+---+-----+---------+
|employee_name|department|state|salary|age|bonus|Increment|
+-------------+----------+-----+------+---+-----+---------+
|        James|     Sales|   NY| 90000| 34|10000|   9500.0|
|      Michael|     Sales|   NY| 86000| 56|20000|   9600.0|
|       Robert|     Sales|   CA| 81000| 30|23000|  10410.0|
|        Maria|   Finance|   CA| 90000| 24|23000|  11490.0|
|        Raman|   Finance|   CA| 99000| 40|24000|  12600.0|
|        Scott|   Finance|   NY| 83000| 36|19000|   9250.0|
|          Jen|   Finance|   NY| 79000| 53|15000|   8650.0|
|         Jeff| Marketing|   CA| 80000| 25|18000|  10140.0|
|        Kumar| Marketing|   NY| 91000| 50|21000|  10150.0|
+-------------+----------+-----+------+---+-----+---------+

